<a href="https://colab.research.google.com/github/cnndabbler/agentic-workflows/blob/main/run_pydantic_ai_with_Ollama_and_DeepSeek_R1_Step_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Pydantic Agents running DeepSeek-R1 using Ollama

In [1]:
!pip install -q pydantic_ai
!pip install -q pydantic
!pip install -q rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.2/223.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.9/252.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.8/210.8 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 271.6/271.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.27.0, but you have google-auth 2.38.0 which is incompatible.


# Initiate Ollama and load the models in the GPU

In [2]:
import subprocess

# Run the Ollama installation script
subprocess.run("curl -fsSL https://ollama.com/install.sh | sh", shell=True, check=True)

# Start the Ollama server in the background
ollama_process = subprocess.Popen(["ollama", "serve"])

# Reasoner model on L4 GPU
# !ollama pull deepseek-r1:14b > /dev/null 2>&1
!ollama pull deepseek-r1:14b

# Reasoner model on A100 GPU
# !ollama pull deepseek-r1:32b > /dev/null 2>&1

# Parser model
# !ollama pull qwen2.5 > /dev/null 2>&1
!ollama pull qwen2.5

!ollama list

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏    0 B/9.0 GB                  pulling manifest 
pulling 6e9f90f02bb3...   0% ▕▏  32 MB/9.0 GB                  pulling manifest 
pu

In [3]:
!ollama list

NAME               ID              SIZE      MODIFIED               
qwen2.5:latest     845dbda0ea48    4.7 GB    Less than a second ago    
deepseek-r1:14b    ea35dfe18182    9.0 GB    About a minute ago        


# Ollama Models

In [4]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

In [5]:
# Initialize models

from pydantic_ai.models.openai import OpenAIModel

# DeepSeekR1
deepseek_reasoner_model = OpenAIModel(
    # 'deepseek-r1:32b',
    'deepseek-r1:14b',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

deepseek_parser_model = OpenAIModel(
    'qwen2.5',
    base_url="http://localhost:11434/v1",
    api_key="whatever",
)

# Pydantic Classes

In [6]:
from typing import List
from pydantic import BaseModel, Field, confloat
import asyncio
from pydantic_ai import Agent, RunContext
from rich.console import Console
from rich.table import Table
from rich import box

# Define the Joke model
class Joke(BaseModel):
    setup: str = Field(..., description="The setup part of the joke")
    punchline: str = Field(..., description="The punchline of the joke")
    category: str = Field(..., description="Category of the joke (e.g., 'pun', 'wordplay', 'dad joke')")
    thinking: str = Field(..., description="the thought process and planning")


In [7]:
# Create Joke Creator Agent
joke_creator = Agent(
    model=deepseek_reasoner_model,
    result_type=str,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Thinking: \n<think>\n[your detailed thought process without changes]\n</think>\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
        # "Thinking: [brief explanation]\n"
    ),
)

# Binding to Pydantic Joke class: Failure

- You will see the following error:
- BadRequestError: Error code: 400 - {'error': {'message': 'registry.ollama.ai/library/deepseek-r1:32b does not support tools', 'type': 'api_error', 'param': None, 'code': None}}

In [8]:
# Create Joke Creator Agent binding result_type to Joke class
joke_creator_with_pydantic_class = Agent(
    model=deepseek_reasoner_model,
    result_type=Joke,
    deps_type=str,  # Takes a string prompt as input
    system_prompt=(
        "You are a joke creator that specializes in creating funny, family-friendly dad jokes. "
        "Your output MUST follow this EXACT format with EXACT field names:\n\n"
        "Thinking: \n<think>\n[your detailed thought process without changes]\n</think>\n\n"
        "Setup: [setup text]\n"
        "Punchline: [punchline text]\n"
        "Category: [category]\n"
    ),
)

NOTE: Only run the cell below to verify that Pydantic Agent does not support DeepSeek-R1

In [9]:
# joke_text = await joke_creator_with_pydantic_class.run("Create a dad joke")
# print("\nRaw joke text:")
# print("-" * 50)
# print(joke_text.data)
# print("-" * 50)

# Using a Parser Agent to successfully bind to Pydantic Class

In [10]:
# Create Parser Agent with 3 retries max
joke_parser = Agent(
    model=deepseek_parser_model,
    result_type=Joke,
    deps_type=str,  # Takes raw text as input
    retries=3,
    system_prompt=(
        "You are a joke parser that extracts components from the input text to create a Joke object. "
        "Your ONLY task is to find and extract these components from the input text:\n"
        "1. The setup (after 'Setup:')\n"
        "2. The punchline (after 'Punchline:')\n"
        "3. The category (after 'Category:')\n"
        "4. The thinking (between <think> tags)\n\n"
        "Return the components in this format:\n"
        "setup=[text after Setup:]\n"
        "punchline=[text after Punchline:]\n"
        "category=[text after Category:]\n"
        "thinking=[text between <think> tags]\n\n"
        "DO NOT generate new content or modify the text in any way.\n"
        "DO NOT include any other text in your response."
    ),
)



In [11]:
# Step 1: Create raw joke text using DeepSeek-R1
print("Creating joke...")
joke_text = await joke_creator.run("Create a dad joke")
if not joke_text:
    raise ValueError("Failed to create joke")
print("\nRaw joke text:")
print("-" * 50)
print(joke_text.data)
print("-" * 50)

Creating joke...

Raw joke text:
--------------------------------------------------
<think>
Alright, so I need to create a dad joke about dictionaries. First off, why dictionaries? Well, they're such a classic topic for puns and wordplay. Dictionaries have words, definitions, parts of speech... So there's plenty of material to work with here.

Let me think about the structure. A good setup should introduce the topic, something like "Why don’t dictionaries argue?" That immediately makes you think about arguments and communication. Now, for the punchline, I need to connect arguing with the nature of dictionaries. Since dictionaries define words but don't have opinions or emotions, they can't argue.

So, putting it together: "Because they’re always right!" Wait, that's a classic setup. But maybe I can make it more specific to dictionaries to add a twist. Thinking deeper—why else can’t dictionaries argue? Because they're reference books, they don't have feelings or desires to disagree. The

In [12]:
# Step 2: Parse the text into a Pydantic model using Qwen2.5
print("\nParsing joke...")
parsed_joke = await joke_parser.run(
    joke_text.data)
if not parsed_joke:
    raise ValueError("Failed to parse joke")
print("\nParsed joke text:")
print("-" * 50)
print(parsed_joke.data)
print("-" * 50)



Parsing joke...

Parsed joke text:
--------------------------------------------------
setup='Why don’t dictionaries argue?' punchline='Because they’re always right!' category='wordplay' thinking="Alright, let me come up with a family-friendly dad joke about dictionaries. Dictionaries are all about words and definitions, so maybe I can play on that somehow. Maybe something about arguments or disagreements? Because 'dictionary' makes me think of 'dictate,' which is like telling someone what to do."
--------------------------------------------------


In [13]:
# Step 3: Display results
console = Console()

# Create assessment table
table = Table(title="Joke Parsed", box=box.ROUNDED)
table.add_column("Property", style="cyan", width=20)
table.add_column("Value", style="yellow")

# Add joke content
table.add_row("Setup", parsed_joke.data.setup)
table.add_row("Punchline", parsed_joke.data.punchline)
table.add_row("Category", parsed_joke.data.category)
table.add_row("","")


# Add thinking process
table.add_row("", "")
table.add_row("[bold]Thinking Process", "")
thinking_lines = parsed_joke.data.thinking.split('\n')
for line in thinking_lines:
    if line.strip():
        table.add_row("", line)

# Print the table
console.print("\n")
console.print(table)

                                                    Joke Parsed                                                    
╭──────────────────────┬──────────────────────────────────────────────────────────────────────────────────────────╮
│ Property             │ Value                                                                                    │
├──────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────┤
│ Setup                │ Why don’t dictionaries argue?                                                            │
│ Punchline            │ Because they’re always right!                                                            │
│ Category             │ wordplay                                                                                 │
│                      │                                                                                          │
│                      │                                                                                          │
│ Thinking Process     │                                                                                          │
│                      │ Alright, let me come up with a family-friendly dad joke about dictionaries. Dictionaries │
│                      │ are all about words and definitions, so maybe I can play on that somehow. Maybe          │
│                      │ something about arguments or disagreements? Because 'dictionary' makes me think of       │
│                      │ 'dictate,' which is like telling someone what to do.                                     │
╰──────────────────────┴──────────────────────────────────────────────────────────────────────────────────────────╯